In [1]:
from typing import TypedDict, Annotated

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model

In [3]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

llm = init_chat_model("google_genai:gemini-2.0-flash")

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [4]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
response = graph.invoke({"messages": [message]})

response['messages']

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='e1af009b-5e72-4be8-a76a-9a0375ed48d8'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--8423d40b-32f4-4d58-a763-54fce64a1085-0', usage_metadata={'input_tokens': 14, 'output_tokens': 3, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]

In [5]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit", "exit"}:
        break
    if state is None:
        state: State = {
            "messages" : [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role":"user", "content": in_message})


    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

Bot: The company founded by Jeff Bezos is **Amazon**.
Bot: I cannot provide you with real-time stock prices. Stock prices fluctuate constantly.

To find the current stock price of Amazon (AMZN), you can check these resources:

*   **Google Finance:** Search "AMZN stock" on Google.
*   **Yahoo Finance:** finance.yahoo.com
*   **Bloomberg:** bloomberg.com
*   **Your Brokerage Account:** If you have a brokerage account, the stock price will be displayed there.
